In [ ]:
from pathlib import Path
from ipynb.fs.full.notebook_viewer import Viewer, Manager
from scipy.spatial import distance
import numpy as np
from tqdm import tqdm
from PIL import Image
from IPython.display import Javascript, display
import ipywidgets as wi
from io import BytesIO
import numpy as np
from script_util import tqdm_load
from nokogiri.working_dir import working_dir
from nokogiri.spylus import spylus

In [ ]:
root = Path("/data/natsuki/training116/00030-v4-mirror-auto4-gamma100-batch64-noaug-resumecustom/011289")
#root = Path("/data/natsuki/training116/00023-white_yc05_yw04-mirror-auto4-gamma10-noaug/021800")
assert root.is_dir()
filelist = list(root.glob("*.png"))
filelist.sort()
print(len(filelist))

In [ ]:
tagnpy = np.load(root/".."/"tag.npy")

In [ ]:
# numericなsortかチェック→違うので変換cmpを用意
for i in range(len(filelist)-1):
    a = filelist[i]
    b = filelist[i+1]
    ai = int(a.stem.replace("seed", ""))
    bi = int(b.stem.replace("seed", ""))
    if ai+1 != bi:
        print(a, b)
        break

In [ ]:
cmpinv = [int(file.stem.replace("seed", "")) for file in filelist]
cmp = {seed:i for i,seed in enumerate(cmpinv)}

In [ ]:
with working_dir("/home/natsuki/illustration2vec"):
    import i2v
    model_feat = i2v.make_i2v_with_chainer("illust2vec_ver200.caffemodel")
    model_tag = i2v.make_i2v_with_chainer("illust2vec_tag_ver200.caffemodel", "tag_list.json")

In [ ]:
fileupload = wi.FileUpload( accept='', multiple=False )
fileupload

In [ ]:
bdata = next(iter(fileupload.value.values()))["content"]
img = Image.open(BytesIO(bdata))

In [ ]:
ID, canvas = spylus.ID_canvas(width=512, height=512)
save_input = wi.Text(layout=wi.Layout(width="10%"))
paste_button = wi.Button(description="paste")
app = wi.VBox((
    save_input,
    paste_button,
    canvas,
))
display(app,
Javascript(f"""{spylus.js}
new (mix(
    White, Save, PasteCenter
))({{
    canvas: document.getElementById("{ID}"),
    save_input: elem('{ID}', 'input', 0),
    paste_button: elem('{ID}', 'button', 0),
}});
"""))

In [ ]:
img = spylus.decode(save_input.value)
type(img)

In [ ]:
target = model_feat.extract_feature([img])[0]
print(target.shape)

In [ ]:
#target = tagnpy[cmp[16188]]

In [ ]:
d = distance.euclidean
#d = distance.cosine
metrix = list()
filelist.sort()
for vec in tqdm(tagnpy):
    metrix.append( d(target, vec) )
tmp = list(zip(metrix, filelist))
tmp.sort()
filelist_sorted = list(zip(*tmp))[1]

In [ ]:
Manager(
    filelist_sorted,
    key="nosort",
    root=root,
    name="len",
    N=27,
    PX=200,
)

In [ ]:
# タグを使った近傍検索
tagpkl = tqdm_load(root/".."/"tag.pkl")

In [ ]:
query = ["pants", "general"] # 短パン
query = ["ia (vocaloid)", "character"]
query = ["green hair", "general"]
query = ["jacket", "general"]
query = ["china dress", "general"]
query = ["shirt", "general"]
query = ["hatsune miku", "character"]
query = ["kagamine len", "character"]
query = ["school uniform", "general"]
metrix = list()
filelist.sort()
for t in tqdm(tagpkl):
    metrix.append( dict(t[query[1]]).get(query[0], -float("inf")) )
    
tmp = list(zip(metrix, filelist))
tmp.sort(reverse=True)
filelist_sorted = list(zip(*tmp))[1]

In [ ]:
Manager(
    filelist_sorted,
    key="nosort",
    root=root,
#    name=f"tag_{query[1]}_{query[0]}",
    name="len",
    N=27,
    PX=200,
)

In [ ]:
Viewer(
    Manager.MANAGED["len"].bag,
    key="nosort",
    N=27,
    PX=200,
    root=root,
)